In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder_path, file_name), sep = ';')
            dataframes[file_name] = df
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    df = df[df['classification'].isin(valid_classes) & df['classification'].notna()]
    discarded_rows = initial_rows - df.shape[0]
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)



    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_comment and type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)
    


    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0

    for file_name, df_pred in predicted_dfs.items():
        df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
        common_index = df_pred_clean.index.intersection(ground_truth_df.index)
        y_pred = df_pred_clean.loc[common_index, 'classification']
        y_true_common = y_true.loc[common_index]
        
        metrics = calculate_metrics(y_true_common, y_pred)
        metrics['file_name'] = file_name
        results.append(metrics)
        
        total_discarded_rows += discarded_rows

    metrics_df = pd.DataFrame(results)
    return metrics_df, total_discarded_rows

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent' ]
ground_truth_file = 'FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = r"C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\log_files\LLAMA3L\csv_from_log_files" #'./experiments_outputs/BOTH'



In [2]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import chardet

def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def try_open_file(file_path, encodings=['utf-8', 'latin-1', 'iso-8859-1']):
    """Tries to open a file with multiple encodings."""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, sep=';', encoding=encoding, on_bad_lines='warn')
            return df, encoding
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to decode file with available encodings: {file_path}")

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            try:
                df, used_encoding = try_open_file(os.path.join(folder_path, file_name))
                dataframes[file_name] = df
                print(f"Loaded {file_name} with encoding {used_encoding}")
            except Exception as e:
                print(f"Failed to load {file_name}: {e}")
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    
    if 'classification_cleaned' in df.columns:
        df = df[df['classification_cleaned'].isin(valid_classes) & df['classification_cleaned'].notna()]
        discarded_rows = initial_rows - df.shape[0]
    else:
        discarded_rows = initial_rows
    
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)
    
    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0
    skipped_files = []

    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' not in df_pred.columns:
            skipped_files.append(file_name)
            continue
        try:
            df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
            common_index = df_pred_clean.index.intersection(ground_truth_df.index)
            y_pred = df_pred_clean.loc[common_index, 'classification_cleaned']
            y_true_common = y_true.loc[common_index]
            
            metrics = calculate_metrics(y_true_common, y_pred)
            metrics['file_name'] = file_name
            results.append(metrics)
            
            total_discarded_rows += discarded_rows
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")
            skipped_files.append(file_name)

    metrics_df = pd.DataFrame(results)

    if skipped_files:
        print("\nSkipped files:")
        for file_name in skipped_files:
            print(file_name)

    return metrics_df, total_discarded_rows

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = 'FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = r"C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\log_files\LLAMA3\csv_from_log_files"

metrics_df, total_discarded_rows = process_files(ground_truth_file, predicted_folder, valid_classes)
print(metrics_df)
print(f"Total discarded rows: {total_discarded_rows}")


FileNotFoundError: [Errno 2] No such file or directory: 'FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'

In [19]:
metrics_df

accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  \
0   0.608303         0.288120      0.323339  0.224635            0.624258   
1   0.590991         0.345238      0.220925  0.215011            0.692321   
2   0.618280         0.294403      0.282240  0.232015            0.631790   
3   0.616071         0.330319      0.337377  0.254379            0.678379   
4   0.595324         0.291854      0.276881  0.235372            0.631566   
5   0.513761         0.266730      0.254809  0.254458            0.600660   
6   0.600719         0.317116      0.326406  0.237957            0.667033   
7   0.545775         0.328884      0.481792  0.242914            0.631986   
8   0.608303         0.246456      0.275358  0.221770            0.577455   
9   0.571949         0.228175      0.213280  0.203728            0.544395   
10  0.596050         0.282061      0.226721  0.224013            0.614964   
11  0.596745         0.269042      0.220272  0.208811            0.603249   
12  0.612613         0.328369      0.277616  0.225453            0.675541   

    recall_weighted  f1_weighted  \
0          0.608303     0.566449   
1          0.590991     0.558363   
2          0.618280     0.579070   
3          0.616071     0.600254   
4          0.595324     0.578695   
5          0.513761     0.544282   
6          0.600719     0.583064   
7          0.545775     0.501363   
8          0.608303     0.567816   
9          0.571949     0.535829   
10         0.596050     0.567462   
11         0.596745     0.555045   
12         0.612613     0.573999   

                                            file_name  
0   _EN_reply_classify_replies_561_MIXTRAL_Hatebas...  
1   _EN_reply_classify_replies_561_MIXTRAL_Hatebas...  
2   _EN_reply_comment_classify_replies_561_MIXTRAL...  
3   _EN_reply_comment_title_classify_replies_561_M...  
4   _EN_reply_comment_title_classify_replies_561_M...  
5   _EN_reply_comment_title_definition_classify_re...  
6   _EN_reply_comment_title_descriptionYT_classify...  
7   _EN_reply_comment_title_transcript_audio_event...  
8   _EN_reply_comment_title_transcript_classify_re...  
9   _EN_reply_comment_title_transcript_description...  
10  _EN_reply_comment_title_transcript_object_clas...  
11  _EN_reply_comment_title_transcript_object_clas...  
12  _EN_reply_title_classify_replies_561_MIXTRAL_H...

In [12]:
metrics_df, total_discarded_rows = process_files(ground_truth_file, predicted_folder, valid_classes)

Loaded _EN_reply_classify_replies_561_MIXTRAL_Hatebase_FINAL.csv with encoding utf-8


ParserError: Error tokenizing data. C error: Expected 3 fields in line 8, saw 4


In [ ]:
metrics_df.to_excel(r"C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\log_files\MIXTRAL\csv_from_log_files\Evaluation_all_MIXTRAL.xlsx")

In [42]:
total_discarded_rows

223

In [44]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder_path, file_name), sep=';')
            dataframes[file_name] = df
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    df = df[df['classification'].isin(valid_classes) & df['classification'].notna()]
    discarded_rows = initial_rows - df.shape[0]
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def get_class_distribution(series):
    """Get the class distribution of a Pandas Series."""
    distribution = series.value_counts(normalize=True).to_dict()
    return distribution

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_comment and type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)
    
    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0

    for file_name, df_pred in predicted_dfs.items():
        df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
        common_index = df_pred_clean.index.intersection(ground_truth_df.index)
        y_pred = df_pred_clean.loc[common_index, 'classification']
        y_true_common = y_true.loc[common_index]
        
        metrics = calculate_metrics(y_true_common, y_pred)
        metrics['file_name'] = file_name
        metrics['model'] = 'LLAMA3' if 'LLAMA3' in file_name else 'MIXTRAL'
        metrics['ground_truth_distribution'] = get_class_distribution(y_true_common)
        metrics['predicted_distribution'] = get_class_distribution(y_pred)
        results.append(metrics)
        
        total_discarded_rows += discarded_rows

    metrics_df = pd.DataFrame(results)
    return metrics_df, total_discarded_rows

def compare_models(ground_truth_file, predicted_folder_1, predicted_folder_2, valid_classes):
    metrics_df_1, discarded_rows_1 = process_files(ground_truth_file, predicted_folder_1, valid_classes)
    metrics_df_2, discarded_rows_2 = process_files(ground_truth_file, predicted_folder_2, valid_classes)

    # Keep only the files that are present in both folders
    common_files = set(metrics_df_1['file_name']).intersection(set(metrics_df_2['file_name']))
    metrics_df_1 = metrics_df_1[metrics_df_1['file_name'].isin(common_files)]
    metrics_df_2 = metrics_df_2[metrics_df_2['file_name'].isin(common_files)]

    # Combine the results for comparison
    combined_metrics_df = metrics_df_1.merge(metrics_df_2, on='file_name', suffixes=('_model_1', '_model_2'))

    # Sort the dataframe by model and then by file name in descending order
    combined_metrics_df = combined_metrics_df.sort_values(by=['model_model_1', 'file_name'], ascending=[True, False])

    return combined_metrics_df, discarded_rows_1, discarded_rows_2

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = 'FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder_1 = './experiments_outputs/MIXTRAL'
predicted_folder_2 = './experiments_outputs/LLAMA3'

# Compare the models
combined_metrics_df, total_discarded_rows_1, total_discarded_rows_2 = compare_models(ground_truth_file, predicted_folder_1, predicted_folder_2, valid_classes)

# Output the results
print("Combined Metrics DataFrame:")
print(combined_metrics_df)
print("\nTotal Discarded Rows (Model 1):", total_discarded_rows_1)
print("Total Discarded Rows (Model 2):", total_discarded_rows_2)


c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Combined Metrics DataFrame:
Empty DataFrame
Columns: [accuracy_model_1, precision_macro_model_1, recall_macro_model_1, f1_macro_model_1, precision_weighted_model_1, recall_weighted_model_1, f1_weighted_model_1, file_name, model_model_1, ground_truth_distribution_model_1, predicted_distribution_model_1, accuracy_model_2, precision_macro_model_2, recall_macro_model_2, f1_macro_model_2, precision_weighted_model_2, recall_weighted_model_2, f1_weighted_model_2, model_model_2, ground_truth_distribution_model_2, predicted_distribution_model_2]
Index: []

[0 rows x 21 columns]

Total Discarded Rows (Model 1): 160
Total Discarded Rows (Model 2): 63


In [46]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder_path, file_name), sep=';')
            dataframes[file_name] = df
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    df = df[df['classification'].isin(valid_classes) & df['classification'].notna()]
    discarded_rows = initial_rows - df.shape[0]
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def class_distribution(y):
    """Calculate the class distribution."""
    return y.value_counts(normalize=True).to_dict()

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_comment and type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)

    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0

    for file_name, df_pred in predicted_dfs.items():
        model_name = "LLAMA3" if "LLAMA3" in file_name else "MIXTRAL"
        df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
        common_index = df_pred_clean.index.intersection(ground_truth_df.index)
        y_pred = df_pred_clean.loc[common_index, 'classification']
        y_true_common = y_true.loc[common_index]

        metrics = calculate_metrics(y_true_common, y_pred)
        metrics['file_name'] = file_name
        metrics['model'] = model_name
        metrics['ground_truth_distribution'] = class_distribution(y_true_common)
        metrics['predicted_distribution'] = class_distribution(y_pred)
        results.append(metrics)

        total_discarded_rows += discarded_rows

    metrics_df = pd.DataFrame(results)
    metrics_df = metrics_df.sort_values(by=['model', 'file_name'], ascending=[True, False])
    return metrics_df, total_discarded_rows

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = 'FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = './experiments_outputs/BOTH'

# Process the files and output the results
metrics_df, total_discarded_rows = process_files(ground_truth_file, predicted_folder, valid_classes)

print("Metrics DataFrame:")
print(metrics_df)
print("\nTotal Discarded Rows:", total_discarded_rows)


c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Metrics DataFrame:
    accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  \
18  0.643243         0.531676      0.762775  0.446570            0.867842   
15  0.693694         0.405462      0.421165  0.298573            0.783187   
13  0.652803         0.515748      0.769654  0.431846            0.877122   
11  0.640288         0.700877      0.725635  0.590637            0.882673   
8   0.632911         0.500475      0.777178  0.417833            0.896328   
6   0.653358         0.533482      0.719502  0.454146            0.882220   
3   0.700368         0.414188      0.467035  0.299097            0.786491   
0   0.648746         0.496638      0.716859  0.382354            0.902311   
19  0.695100         0.403235      0.419038  0.295945            0.782866   
17  0.700717         0.429576      0.471357  0.304545            0.814425   
16  0.698182         0.399663      0.418898  0.293074            0.777925   
14  0.701275         0.428262      0.471804  0.306563    

c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [53]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            df = pd.read_csv(os.path.join(folder_path, file_name), sep=';')
            dataframes[file_name] = df
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    df = df[df['classification'].isin(valid_classes) & df['classification'].notna()]
    discarded_rows = initial_rows - df.shape[0]
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def class_distribution(y):
    """Calculate the class distribution."""
    return y.value_counts(normalize=True).to_dict()

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_comment and type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)

    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0
    all_common_indices = None

    # First pass to find common indices
    for file_name, df_pred in predicted_dfs.items():
        df_pred_clean, _ = clean_dataframe(df_pred, valid_classes)
        if all_common_indices is None:
            all_common_indices = df_pred_clean.index
        else:
            all_common_indices = all_common_indices.intersection(df_pred_clean.index)

    # Ground truth distribution using common indices
    y_true_common = y_true.loc[all_common_indices]
    ground_truth_dist = class_distribution(y_true_common)

    for file_name, df_pred in predicted_dfs.items():
        model_name = "LLAMA3" if "LLAMA3" in file_name else "MIXTRAL"
        df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
        y_pred = df_pred_clean.loc[all_common_indices, 'classification']

        metrics = calculate_metrics(y_true_common, y_pred)
        metrics['file_name'] = file_name
        metrics['model'] = model_name
        metrics['ground_truth_distribution'] = ground_truth_dist
        metrics['predicted_distribution'] = class_distribution(y_pred)
        results.append(metrics)

        total_discarded_rows += discarded_rows

    metrics_df = pd.DataFrame(results)
    metrics_df = metrics_df.sort_values(by=['model', 'file_name'], ascending=[True, True])
    return metrics_df, total_discarded_rows

def highlight_max(data, color='lightgreen'):
    '''
    highlight the maximum in a Series or DataFrame
    '''
    attr = f'background-color: {color}'
    # find the relevant values
    is_max = data >= data.max() * 0.99
    return [attr if v else '' for v in is_max]

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = 'FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = './experiments_outputs/BOTH'

# Process the files and output the results
metrics_df, total_discarded_rows = process_files(ground_truth_file, predicted_folder, valid_classes)

# Separate DataFrames for each model
llama3_df = metrics_df[metrics_df['model'] == 'LLAMA3']
mixtral_df = metrics_df[metrics_df['model'] == 'MIXTRAL']

# Highlight the highest values for F1 scores and accuracy in light colors
llama3_df_styled = llama3_df.style.apply(highlight_max, subset=['accuracy', 'f1_macro', 'f1_weighted'], axis=0)
mixtral_df_styled = mixtral_df.style.apply(highlight_max, subset=['accuracy', 'f1_macro', 'f1_weighted'], axis=0)

# Save the styled DataFrames to separate sheets in an Excel file
excel_file = 'model_comparison_metrics2.xlsx'
with pd.ExcelWriter(excel_file) as writer:
    llama3_df_styled.to_excel(writer, sheet_name='LLAMA3', index=False)
    mixtral_df_styled.to_excel(writer, sheet_name='MIXTRAL', index=False)

print(f"Metrics DataFrame saved to {excel_file}")
print("\nTotal Discarded Rows:", total_discarded_rows)


c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

Metrics DataFrame saved to model_comparison_metrics2.xlsx

Total Discarded Rows: 306


c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f

In [3]:
import pandas as pd
import numpy as np
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import chardet

def detect_file_encoding(file_path):
    """Detects the encoding of a file."""
    with open(file_path, 'rb') as f:
        raw_data = f.read(10000)  # Read a sample of the file
    result = chardet.detect(raw_data)
    return result['encoding']

def load_ground_truth(file_path):
    """Load the ground truth data from an Excel file."""
    return pd.read_excel(file_path)

def try_open_file(file_path, encodings=['utf-8', 'latin-1', 'iso-8859-1']):
    """Tries to open a file with multiple encodings."""
    for encoding in encodings:
        try:
            df = pd.read_csv(file_path, sep=';', encoding=encoding, on_bad_lines='warn')
            return df, encoding
        except UnicodeDecodeError:
            continue
    raise UnicodeDecodeError(f"Unable to decode file with available encodings: {file_path}")

def load_predicted_dataframes(folder_path):
    """Load all CSV files from a folder and return a dictionary of DataFrames."""
    dataframes = {}
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            try:
                df, used_encoding = try_open_file(os.path.join(folder_path, file_name))
                dataframes[file_name] = df
                print(f"Loaded {file_name} with encoding {used_encoding}")
            except Exception as e:
                print(f"Failed to load {file_name}: {e}")
    return dataframes

def clean_dataframe(df, valid_classes):
    """Clean the DataFrame by excluding invalid or empty classifications."""
    initial_rows = df.shape[0]
    
    if 'classification_cleaned' in df.columns:
        df = df[df['classification_cleaned'].isin(valid_classes) & df['classification_cleaned'].notna()]
        discarded_rows = initial_rows - df.shape[0]
    else:
        discarded_rows = initial_rows
    
    return df, discarded_rows

def calculate_metrics(y_true, y_pred):
    """Calculate accuracy, precision, recall, and F1 score (both weighted and macro)."""
    metrics = {
        'accuracy': accuracy_score(y_true, y_pred),
        'precision_macro': precision_score(y_true, y_pred, average='macro'),
        'recall_macro': recall_score(y_true, y_pred, average='macro'),
        'f1_macro': f1_score(y_true, y_pred, average='macro'),
        'precision_weighted': precision_score(y_true, y_pred, average='weighted'),
        'recall_weighted': recall_score(y_true, y_pred, average='weighted'),
        'f1_weighted': f1_score(y_true, y_pred, average='weighted')
    }
    return metrics

def calculate_class_distribution(series):
    """Calculate class distribution for a series of classifications."""
    distribution = (series.value_counts(normalize=True) * 100).round(2).to_dict()
    return distribution

def process_files(ground_truth_file, predicted_folder, valid_classes):
    ground_truth_df = load_ground_truth(ground_truth_file)
    predicted_dfs = load_predicted_dataframes(predicted_folder)

    # Mapping for new class names
    class_mapping = {
        "0. appropriate": "appropriate",
        "1. inappropriate": "inappropriate",
        "2. offensive": "offensive",
        "3. violent": "violent"
    }

    # Rename classes in type_reply
    ground_truth_df["type_reply"] = ground_truth_df["type_reply"].map(class_mapping)
    
    y_true = ground_truth_df['type_reply']
    results = []

    total_discarded_rows = 0
    skipped_files = []
    rows_used_dict = {}
    rows_not_used_dict = {}

    common_index = y_true.index

    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' not in df_pred.columns:
            skipped_files.append(file_name)
            continue
        try:
            df_pred_clean, discarded_rows = clean_dataframe(df_pred, valid_classes)
            common_index = common_index.intersection(df_pred_clean.index)
            total_discarded_rows += discarded_rows
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")
            skipped_files.append(file_name)

    # Remove rows with 'none' classification in any file
    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' in df_pred.columns:
            df_pred_clean = df_pred.loc[common_index]
            if 'none' in df_pred_clean['classification_cleaned'].values:
                common_index = common_index.difference(df_pred_clean.index[df_pred_clean['classification_cleaned'] == 'none'])

    # Process remaining files with common index
    for file_name, df_pred in predicted_dfs.items():
        if 'classification_cleaned' not in df_pred.columns or file_name in skipped_files:
            continue
        try:
            df_pred_clean = df_pred.loc[common_index]
            y_pred = df_pred_clean['classification_cleaned']
            y_true_common = y_true.loc[common_index]
            
            metrics = calculate_metrics(y_true_common, y_pred)
            metrics['file_name'] = file_name

            # Calculate class distributions
            ground_truth_distribution = calculate_class_distribution(y_true_common)
            predicted_distribution = calculate_class_distribution(y_pred)
            metrics['ground_truth_distribution'] = str(ground_truth_distribution)
            metrics['predicted_distribution'] = str(predicted_distribution)

            results.append(metrics)

            # Record used and not used row indices
            rows_used_dict[file_name] = common_index.tolist()
            rows_not_used_dict[file_name] = [idx for idx in df_pred.index if idx not in common_index]
        except Exception as e:
            print(f"Failed to process {file_name}: {e}")
            skipped_files.append(file_name)

    metrics_df = pd.DataFrame(results)

    if skipped_files:
        print("\nSkipped files:")
        for file_name in skipped_files:
            print(file_name)

    return metrics_df, total_discarded_rows, rows_used_dict, rows_not_used_dict

# Define valid classes and file paths
valid_classes = ['appropriate', 'inappropriate', 'offensive', 'violent']
ground_truth_file = r'C:\Users\aboeck\Documents\ICMT\MASTERARBEIT_DI\backup_03_07\EVALUATION\FINAL_IMSyPP_EN_561_PREPROCESSED_no_restricted_videos_features_2ktoken_w_prompt_u_desc_audio_object_with_video_and_reply_metadata_no_missing.xlsx'
predicted_folder = r"./"

metrics_df, total_discarded_rows, rows_used_dict, rows_not_used_dict = process_files(ground_truth_file, predicted_folder, valid_classes)
print(metrics_df)
print(f"Total discarded rows: {total_discarded_rows}")

# Add columns for rows used and not used
metrics_df['rows_used'] = metrics_df['file_name'].map(rows_used_dict)
metrics_df['rows_not_used'] = metrics_df['file_name'].map(rows_not_used_dict)

# Save the metrics DataFrame as an Excel file
output_file = 'metrics_results.xlsx'
metrics_df.to_excel(output_file, index=False)
print(f"Metrics saved to {output_file}")


Loaded TEST.csv with encoding utf-8
Loaded _EN_reply_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTE_here.csv with encoding utf-8
Loaded _EN_reply_classify_replies_561_LLAMA3_Hatebase_FINAL_FINAL_SECOND.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTEeeeeee.csv with encoding utf-8
Loaded _EN_reply_comment_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTEeeeeee_SECOND.csv with encoding utf-8
Loaded _EN_reply_comment_title_classify_replies_561_LLAMA3_Hatebase_FINAL_AAAAAAAABSOLUTEeeeeee_SECOND_new.csv with encoding utf-8
Loaded _EN_reply_comment_title_classify_replies_561_LLAMA3_Hatebase_FINAL_SECOND.csv with encoding utf-8
Loaded _EN_reply_comment_title_descriptionYT_classify_replies_561_LLAMA3_Hatebase_FINAL.csv with encoding utf-8
Loaded _EN_reply_comment_title_description_classify_replies_561_LLAMA3_Hatebase_FINAL.c

c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

    accuracy  precision_macro  recall_macro  f1_macro  precision_weighted  \
0   0.363636         0.666667      0.255556  0.345238            1.000000   
1   0.363636         0.416667      0.255556  0.305556            0.613636   
2   0.363636         0.583333      0.255556  0.317460            0.886364   
3   0.363636         0.333333      0.255556  0.265152            0.500000   
4   0.454545         0.666667      0.311111  0.416667            1.000000   
5   0.454545         0.583333      0.311111  0.388889            0.886364   
6   0.545455         0.583333      0.366667  0.450000            0.863636   
7   0.363636         0.500000      0.255556  0.333333            0.727273   
8   0.454545         0.583333      0.311111  0.388889            0.886364   
9   0.363636         0.666667      0.255556  0.345238            1.000000   
10  0.454545         0.555556      0.311111  0.398148            0.818182   
11  0.454545         0.555556      0.311111  0.398148            0.818182   

c:\Users\aboeck\AppData\Local\anaconda3\envs\new_MA\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
